In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

# 1. 환경 설정
plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['axes.unicode_minus'] = False
warnings.filterwarnings('ignore')

# 2. 데이터 로드
df_info = pd.read_csv('./data/df_info2.csv', encoding='utf-8-sig')
df_sales = pd.read_csv('./data/df_sales2.csv', encoding='utf-8-sig')
df_customer = pd.read_csv('./data/df_customer2.csv', encoding='utf-8-sig')

# 3. 기본 전처리 (컬럼 정렬 및 날짜 변환)
cols_order = ['가맹점구분번호', '가맹점명', '상권', '가맹점주소', '가맹점주소_세부', '업종', '프랜차이즈여부', '개설일', '폐업일']
df_info = df_info[cols_order]
df_info['개설일'] = pd.to_datetime(df_info['개설일'], errors='coerce')
df_info['폐업일'] = pd.to_datetime(df_info['폐업일'], errors='coerce')

# 4. 업종 대분류 정의 (Classification Rules)
classification_rules = {
    '식사': ['한식', '일식', '중식', '분식', '치킨', '햄버거', '피자', '샌드위치', '도시락', '스테이크', '양식', '세계요리', '구내식당', '기사식당'],
    '주점/유흥': ['호프', '주점', '꼬치', '와인', '룸살롱', '포차', '이자카야'],
    '카페/디저트': ['카페', '마카롱', '차', '탕후루', '커피', '베이커리', '아이스크림', '도너츠', '와플', '주스'],
    '비외식소매': ['건강', '건어물', '농산물', '담배', '떡', '미곡', '반찬', '수산', '식료', '와인샵', '유제품', '인삼', '주류', '청과', '축산']
}

def classify_major_category(industry, rules):
    if not isinstance(industry, str): return '기타'
    for category, keywords in rules.items():
        if any(k in industry for k in keywords):
            return category
    return '기타'

df_info['대분류'] = df_info['업종'].apply(lambda x: classify_major_category(x, classification_rules))

print("===== 대분류 작업 완료 =====")
print(df_info['대분류'].value_counts())

===== 대분류 작업 완료 =====
대분류
식사        1998
카페/디저트     809
비외식소매      657
기타         401
주점/유흥      318
Name: count, dtype: int64


In [11]:
def hierarchical_imputation(df_target, df_meta, target_cols, group_keys_list):
    """
    계층적 결측치 대체 함수
    :param df_target: 결측치를 채울 데이터프레임 (Sales or Customer)
    :param df_meta: 메타 정보가 있는 데이터프레임 (Info)
    :param target_cols: 채울 컬럼 리스트
    :param group_keys_list: 대체 기준 그룹 리스트 (우선순위 순)
    """
    # 1. 이상치 처리 및 병합
    df_target.replace(-999999.9, np.nan, inplace=True)
    df_merged = pd.merge(df_target, df_meta[['가맹점구분번호', '상권', '업종', '대분류']], on='가맹점구분번호', how='left')
    
    for col in target_cols:
        # Level 1: 가게별(가맹점구분번호) 평균/최빈값 (이미 존재하는 경우)
        if df_target is df_sales: # Sales는 평균
             df_merged[col].fillna(df_merged.groupby('가맹점구분번호')[col].transform('mean'), inplace=True)
        else: # Customer는 최빈값
             df_merged[col].fillna(df_merged.groupby('가맹점구분번호')[col].transform(lambda x: x.mode()[0] if not x.mode().empty else np.nan), inplace=True)

        # Level 2~N: 지정된 그룹 키로 순차적 대체
        for keys in group_keys_list:
            fill_val = df_merged.groupby(keys)[col].transform(lambda x: x.mean() if df_target is df_sales else (x.mode()[0] if not x.mode().empty else np.nan))
            df_merged[col].fillna(fill_val, inplace=True)
        
        # Level Final: 전체 평균/최빈값
        global_val = df_merged[col].mean() if df_target is df_sales else df_merged[col].mode()[0]
        df_merged[col].fillna(global_val, inplace=True)
        
        # 원본 업데이트
        df_target[col] = df_merged[col]
    
    return df_target

# --- 1. Sales 결측치 처리 ---
# 배달비율, 취소율은 간단하게 처리
df_sales['배달매출금액 비율'].fillna(0, inplace=True) # 2차: 0
df_sales['취소율 구간'].fillna(df_sales['취소율 구간'].mean(), inplace=True) # 2차: 전체평균

# '해지 가맹점 비중' 계층적 대체
hierarchical_imputation(
    df_sales, df_info, ['동일 상권 내 해지 가맹점 비중'], 
    [['기준년월', '상권'], ['상권'], ['대분류']]
)

# --- 2. Customer 결측치 처리 ---
cust_cols = [c for c in df_customer.columns if '비중' in c or '비율' in c]
hierarchical_imputation(
    df_customer, df_info, cust_cols,
    [['기준년월', '상권', '업종'], ['기준년월', '상권', '대분류']]
)

print("===== 결측치 처리 완료 =====")

===== 결측치 처리 완료 =====


In [12]:
# 1. 군집화용 데이터 준비 (Info + Sales + Customer)
df_merged_cluster = pd.merge(
    df_sales[['가맹점구분번호', '기준년월', '객단가 구간', '배달매출금액 비율']],
    df_customer, on=['가맹점구분번호', '기준년월'], how='inner'
)
df_final_cluster = pd.merge(df_info[['가맹점구분번호', '대분류', '업종']], df_merged_cluster, on='가맹점구분번호', how='left')

# 2. 업종별 프로필 생성 (평균)
features = ['객단가 구간'] # 형이 선택한 피처 (필요시 추가)
df_siksa = df_final_cluster[df_final_cluster['대분류'] == '식사']
df_industry_profiles = df_siksa.groupby('업종')[features].mean()

# 3. K-Means Clustering
# 데이터 정규화
scaler = StandardScaler()
scaled_profiles = scaler.fit_transform(df_industry_profiles)

# 최적 군집 수 (K=4) 설정 및 학습
k = 4
kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
clusters = kmeans.fit_predict(scaled_profiles)

# 4. 결과 매핑 ('식사_그룹_0', '식사_그룹_1'...)
industry_to_cluster = pd.Series(clusters, index=df_industry_profiles.index)
new_labels = df_info.loc[df_info['대분류'] == '식사', '업종'].map(industry_to_cluster)
df_info.loc[df_info['대분류'] == '식사', '대분류'] = '식사_그룹_' + new_labels.dropna().astype(int).astype(str)

print("===== 식사 그룹 세분화 완료 =====")
print(df_info['대분류'].value_counts())

===== 식사 그룹 세분화 완료 =====
대분류
식사_그룹_1    888
식사_그룹_0    813
카페/디저트     809
비외식소매      657
기타         401
주점/유흥      318
식사_그룹_2    290
식사_그룹_3      7
Name: count, dtype: int64


In [13]:
# 1. 12개월 이상 데이터가 있는 가게 식별
store_counts = df_sales['가맹점구분번호'].value_counts()
valid_stores = store_counts[store_counts >= 12].index

# 2. 필터링 적용 (VIP 리스트만 남기기)
print(f"필터링 전: {len(df_info)}개")
df_info = df_info[df_info['가맹점구분번호'].isin(valid_stores)].copy()
df_sales = df_sales[df_sales['가맹점구분번호'].isin(valid_stores)].copy()
df_customer = df_customer[df_customer['가맹점구분번호'].isin(valid_stores)].copy()
print(f"필터링 후: {len(df_info)}개 (12개월 이상 운영)")

필터링 전: 4183개
필터링 후: 3623개 (12개월 이상 운영)


In [14]:
# 기준일 설정
ref_date = pd.to_datetime('2025-08-01')

# 운영 개월 수 계산
df_info['25년기준_운영개월수'] = (ref_date.year - df_info['개설일'].dt.year) * 12 + \
                            (ref_date.month - df_info['개설일'].dt.month)

# 구간화 (Binning)
bins = [-1, 24, 48, 100, float('inf')]
labels = ['1. 초기', '2. 성장기(죽음의계곡)', '3. 성숙기', '4. 장기/쇠퇴기']
df_info['영업기간_구간'] = pd.cut(df_info['25년기준_운영개월수'], bins=bins, labels=labels)

print("===== 운영 기간 피처 생성 완료 =====")
print(df_info[['가맹점명', '영업기간_구간']].head())

===== 운영 기간 피처 생성 완료 =====
   가맹점명    영업기간_구간
0  성우**  4. 장기/쇠퇴기
1  대보**  4. 장기/쇠퇴기
2  대용**  4. 장기/쇠퇴기
3  통일**  4. 장기/쇠퇴기
4  한울**  4. 장기/쇠퇴기


In [ ]:
# 1. 월별 -> 분기별 Pivot
df_sales['기준년월'] = pd.to_datetime(df_sales['기준년월'], errors='coerce')
df_sales['연도'] = df_sales['기준년월'].dt.year
df_sales['분기'] = df_sales['기준년월'].dt.quarter
quarterly_sales = df_sales.groupby(['가맹점구분번호', '연도', '분기'])['매출금액 구간'].mean().unstack(['연도', '분기'])
quarterly_sales.columns = [f"매출금액_{c[0]}년_{c[1]}분기" for c in quarterly_sales.columns]

# 결측치 보간 (앞뒤로 채우기)
quarterly_sales.fillna(method='ffill', axis=1, inplace=True)
quarterly_sales.fillna(method='bfill', axis=1, inplace=True)

# 2. 필살기 지표 생성 (모멘텀, 안정성, 위험가속도)
cols_23 = [c for c in quarterly_sales.columns if '23년' in c]
cols_24 = [c for c in quarterly_sales.columns if '24년' in c]

# 성장 모멘텀 (24년 평균 - 23년 평균)
quarterly_sales['성장_모멘텀'] = quarterly_sales[cols_24].mean(axis=1) - quarterly_sales[cols_23].mean(axis=1)

# 안정성 지수 (전체 기간 표준편차)
quarterly_sales['안정성_지수'] = quarterly_sales.std(axis=1)

# 위험 가속도 (24년 하반기 - 24년 상반기)
cols_24_h1 = [c for c in cols_24 if '1분기' in c or '2분기' in c]
cols_24_h2 = [c for c in cols_24 if '3분기' in c or '4분기' in c]
quarterly_sales['위험_가속도'] = quarterly_sales[cols_24_h2].mean(axis=1) - quarterly_sales[cols_24_h1].mean(axis=1)

# 3. 경쟁력 지표 (동일 업종 대비 순위 등) 변화율
comp_features = ['동일 업종 매출금액 비율', '동일 업종 내 매출 순위 비율', '유니크 고객 수 구간', '객단가 구간']
yearly_comp = df_sales.groupby(['가맹점구분번호', '연도'])[comp_features].mean().unstack('연도')

comp_report = pd.DataFrame(index=yearly_comp.index)
for col in comp_features:
    # 24년 값과 23->24 변화율 계산
    comp_report[f'24년_평균_{col}'] = yearly_comp[(col, 2024)]
    comp_report[f'23_24_{col}_변화'] = yearly_comp[(col, 2024)] - yearly_comp[(col, 2023)]

# 최종 병합
df_sales_final = pd.merge(quarterly_sales, comp_report, left_index=True, right_index=True, how='left')

print("===== 매출 트렌드 분석 완료 =====")
print(f"최종 피처 수: {df_sales_final.shape[1]}개")

===== 매출 트렌드 분석 완료 =====
최종 피처 수: 19개


In [ ]:
# 인덱스 리셋 후 저장
df_sales_final.reset_index(inplace=True)

# 저장
df_info.to_csv('./data/df_info99.csv', encoding='utf-8-sig', index=False)
df_sales_final.to_csv('./data/df_sales99.csv', encoding='utf-8-sig', index=False)
df_customer.to_csv('./data/df_customer99.csv', encoding='utf-8-sig', index=False)

print("모든 작업 완료! 파일 생성됨: df_info99.csv, df_sales99.csv, df_customer99.csv")

🎉 모든 작업 완료! 파일 생성됨: df_info99.csv, df_sales99.csv, df_customer99.csv
